# HuggingFace with GPU
###### 참고
- [Hugging Face - Training on One GPU](https://huggingface.co/docs/transformers/perf_train_gpu_one)
- [Hugging Face - Inference on One GPU](https://huggingface.co/docs/transformers/perf_infer_gpu_one)

## Efficient Training on a Single GPU

더 적은 메모리 사용, 모델 학습 속도 상승, Trainer와 Accelerate가 통합되는 과정을 살펴본다.

아래의 각 method들은 속도 또는 메모리 사용 면에서 향상된다.

| Method | Speed | Memory | 비고 |
| --- | --- | --- |
| Gradient accumulation | No | Yes | 높은 batch size의 부하를 견디기 위해 모든 batch가 gradient를 Global Gradient에 누적시킨 뒤 한 번의 forward pass와 back propagation을 통해 전달 |
| Gradient checkpointing | No | Yes | 모델 학습 시 모든 노드의 가중치를 저장하지 않고 check point의 가중치만 저장하여 속도가 느린 대신 메모리 사용량이 줄어듦 |
| Mixed precision training | Yes | (No) | FP(Float Precision)16과 FP32를 혼용, FW / BW propagation은 FP16, 가중치를 업데이트하는 과정에서 다시 FP32로 변환해 메모리 사용을 줄이고 변환 과정의 오차로 인한 loss값도 줄임 |
| Batch size | Yes | Yes | - |
| Optimizer choice | Yes | Yes | - |
| DataLoader | Yes | No | - |
| Deepspeed Zero | No | Yes | 분산 학습 과정에서의 불필요한 메모리 중복을 제거하여 동시에 학습 가능한 파라미터의 수를 크레 늘릴 수 있는 새로운 병렬 최적화 도구 |

### Libraries
`pip install transformers datasets accelerate nvidia-ml-py3`

- accelerate: 같은 PyTorch 코드라도 4줄만 추가해 더 쉽고 효율적으로 사용할 수 있도록 만들어주는 라이브러리
- nvidia-ml-py3: 모델의 메모리 사용량을 파이썬에서 모니터링할 수 있는 라이브러리. 터미널의 nvidia-smi와 유사

In [1]:
# dummy data

import numpy as np
from datasets import Dataset

seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, dataset_size)
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [2]:
# GPU 사용율 및 Trainer를 사용한 훈련 실행에 대한 요약 통계

from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

print_gpu_utilization()

GPU memory occupied: 343 MB.


In [3]:
# Load Model bert-large-uncased model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-large-uncased").to("cuda")
print_gpu_utilization()

bin C:\Users\jongg\PycharmProjects\HuggingFaceKoLLaMa13b\venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory occupied: 1734 MB.


In [4]:
# default training args

default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none"
}

## Training 함수

In [14]:
from torch import nn
from transformers import TrainingArguments, Trainer, logging
from transformers.trainer_pt_utils import get_parameter_names
from transformers.training_args import OptimizerNames
from typing import Union
import bitsandbytes as bnb
import torch.cuda
import gc

logging.set_verbosity_error()  # error log activate


def get_training_args(
        per_device_train_batch_size: int = 8,
		gradient_accumulation_steps: int = 1,
		gradient_checkpointing: bool = False,
		optim: Union[OptimizerNames, str] = "adamw_hf",
		float_precision: str = None,
		deepspeed: str = None,
) -> TrainingArguments:
	training_args = TrainingArguments(
		per_device_train_batch_size = per_device_train_batch_size,
		gradient_accumulation_steps = gradient_accumulation_steps,
		gradient_checkpointing = gradient_checkpointing,
		optim = optim,
        deepspeed = deepspeed,
		**default_args)
	
	if float_precision == "fp16":
		training_args.fp16 = True
	elif float_precision == "bf16":
		training_args.bf16 = True
	elif float_precision == "tf32":
		training_args.tf32 = True
	
	return training_args

def clean_training(
		per_device_train_batch_size: int = 8,
		gradient_accumulation_steps: int = 1,
		gradient_checkpointing: bool = False,
		optim: Union[OptimizerNames, str] = "adamw_hf",
		float_precision: str = None,
		deepspeed: str = None,
        training: bool = True,
        use_bnb: bool = False
):
    if training:
        training_args = get_training_args(
	        per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            gradient_checkpointing = gradient_checkpointing,
            optim = optim,
            float_precision = float_precision,
            deepspeed = deepspeed
	    )
        
        if float_precision == "fp16":
            training_args.fp16 = True
        elif float_precision == "bf16":
            training_args.bf16 = True
        elif float_precision == "tf32":
            training_args.tf32 = True
            
        # trainer = get_trainer(args = training_args, use_bnb = use_bnb)
        
        if use_bnb:
            decay_parameters = get_parameter_names(model, [nn.LayerNorm])
            decay_parameters = [name for name in decay_parameters if "bias" not in name]
            optimizer_grouped_parameters = [
                {
                    "params": [p for n, p in model.named_parameters() if n in decay_parameters],
                    "weight_decay": training_args.weight_decay
                },
                {
                    "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
                    "weight_decay": 0.0
                }
            ]
            adam_bnb_optim = bnb.optim.AdamW8bit(
                optimizer_grouped_parameters,
                betas = (training_args.adam_beta1, training_args.adam_beta2),
                eps = training_args.adam_epsilon,
                lr = training_args.learning_rate
            )
            trainer = Trainer(model = model, args = training_args, train_dataset = ds, optimizers = (adam_bnb_optim, None))
            
            del decay_parameters, optimizer_grouped_parameters, adam_bnb_optim
        
        else:
            trainer = Trainer(model = model, args = training_args, train_dataset = ds)
        
        result = trainer.train()
        print_summary(result)
        
        del trainer, training_args, result
    
    gc.collect()
    torch.cuda.empty_cache()
    print()
    print("_______________________torch.cuda.empty_cache()_______________________")
    print(f"Allocated GPU memory: {torch.cuda.memory_allocated('cuda:0') / (1024 ** 3):.2f}GB")
    print(f"Reserved GPU memory: {torch.cuda.memory_reserved('cuda:0') / (1024 ** 3):.2f}GB")


clean_training(training = False)


_______________________torch.cuda.empty_cache()_______________________
Allocated GPU memory: 1.25GB
Reserved GPU memory: 1.26GB


## Vanila Training

default_args, batch_size = 4, Trainer를 통해 모델 학습

In [6]:
clean_training(
    per_device_train_batch_size = 4
)

C:\Users\jongg\PycharmProjects\HuggingFaceKoLLaMa13b\venv\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'train_runtime': 110.7153, 'train_samples_per_second': 4.624, 'train_steps_per_second': 1.156, 'train_loss': 0.029618393629789352, 'epoch': 1.0}
Time: 110.72
Samples/second: 4.62
GPU memory occupied: 12239 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


batch_size가 4인데 GPU의 전체 메모리 12GB를 거의 가득 채우고 있다.

배치 크기가 클수록 모델 수렴 속도가 빨라지거나 최종 성능이 향상될 수 있다.

따라서 이상적으로는 GPU 제한이 아닌 모델의 요구사항에 맞게 배치 크기를 조정하고 싶으나 모델의 크기보다 훨씬 더 많은 메모리를 사용하고 있다.

그 이유를 더 잘 이해하기 위해 모델의 작동 및 메모리 요구사항을 살펴보자

# Model`s Operations

Transformers 아키텍처는 아래 3개 메인 그룹의 operation을 포함한다.

1. Tensor Contractions, 텐소 축소
    - Multi-Head Attention의 선형 레이어와 컴포넌트들은 모두 행렬X행렬의 배치 작업을 수행하는데 가장 compute-intensive한 부분이다.
2. Statistical Normalizations
    - Softmax와 레이어 정규화는 tensor contractions보다 덜 compute-intensive하지만 하나 이상의 축소 연산을 포함하며 그 결과는 맵을 통해 적용된다.
3. Element-wise Operators
    - biases, dropout, activations, residual connections는 그 외 가장 compute-intensive하지 않은 연산들이다.


# Model`s Memory

모델이 학습하는 동안 아래와 같은 수많은 컴포넌트가 사용되어 모델 크기에 비해 훨씬 많은 메모리가 사용된다.
1. 모델 가중치
2. optimizer states
3. gradient
4. forward activations saved for gradient computation
5. temporary buffers
6. functionality-specific memory

AdamW를 사용하여 혼합정밀도를 사용하는 기존 모델은 학습시마다 파라미터당 18바이트의 메모리가 필요하다. 추론을 위해 optimizer states와 gradient가 없으므로 이를 빼면 혼합 정밀도 추론을 위한 모델 파라미터당 6바이트와 활성화 메모리로 끝난다.
1. Model Weights
    - FP32 훈련시 파라미터당 4바이트
    - FP16 & FP32 혼합 정밀도 훈련시 파라미터당 6바이트
2. Optimizer States
    - 기존 AdamW와 같이 2state 유지하는 옵티마이저 사용시 파라미터당 8바이트
    - bitsandbytes와 같은 8-bit 옵티마이저 사용시 파라미터당 2바이트
    - momentum을 사용하는 SGD 옵티마이저와 같이 1state만 유지하는 경우 파라미터당 4바이트
3. Gradients:
    - gradient는 항상 FP32, 파라미터당 4바이트 할당
4. Forward Activations:
    - sequence length, hidden size, batch size 등 많은 요소에 의존
5. Temporary Memory:
    - 훈련시 사용되는 임시 변수들
6. Fuctionality-specific memory:
    - 특정 함수의 메모리 사용량
    - ex.) generating text는 beam search를 사용하는데 input과 output의 복사본을 여러개 사용한다.

## Gradient Accumulation

gradient accumulation step 동안 mini-batch 수행 후 gradient를 global gradient에 축적한다.

훈련시간은 느려지나 훈련의 accuracy는 증가한다.

In [7]:
# Trainer에게 TrainingArguments 객체를 전달할 때 gradient_accumulation_steps 값을 주는 것으로 쉽게 사용 사능하다.

clean_training(
	per_device_train_batch_size = 1,
	gradient_accumulation_steps = 4
)

{'train_runtime': 119.3135, 'train_samples_per_second': 4.291, 'train_steps_per_second': 1.073, 'train_loss': 1.2572745617944747e-06, 'epoch': 1.0}
Time: 119.31
Samples/second: 4.29
GPU memory occupied: 8357 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


#### training 결과

- 훈련시간 증가
- 메모리 사용량 감소

> 64개 batch 훈련이 필요하다면 batch size = 4, gradient accumulation size = 16이 같은 batch 크기를 가지면서 메모리는 훨씬 적게 사용한다.

## Gradient Checkpointing

large model 훈련시 메모리가 부족할 수 있다.

TrainingArguments에 gradient_checkpointing = true 값을 주고 더 효율적은 훈련이 가능하다.

In [10]:
clean_training(
	per_device_train_batch_size = 1,
	gradient_accumulation_steps = 4,
	gradient_checkpointing = True
)

{'train_runtime': 154.3928, 'train_samples_per_second': 3.316, 'train_steps_per_second': 0.829, 'train_loss': 2.8638167393069125e-08, 'epoch': 1.0}
Time: 154.39
Samples/second: 3.32
GPU memory occupied: 6872 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


#### training 결과

마찬가지로 gradient_checkpointing = False와 비교했을 때, 훈련시간은 증가했으나 메모리 사용량은 감소했다.

일반적으로 약 20% 정도 느려진다고 한다.

아래 혼합 정밀도 훈련에서 속도를 다시 보장할 수 있다

## Floating Data Types

혼합 정밀도 훈련의 핵심은 모든 변수가 32비트에 저장될 필요는 없다는 것이다.
- fp32(float32)
- fp16(float16)
- bf16(bfloat16)
- tf32(CUDA internal data type)

![](assets/tf32-bf16-fp16-fp32.png)

In [13]:
# FP16 Training
"""
- batch 4
- gradient accumulation 적용
- gradient checkpoint 사용
"""

clean_training(
	per_device_train_batch_size = 1,
	gradient_accumulation_steps = 4,
	gradient_checkpointing = True,
	float_precision = "fp16"
)

{'train_runtime': 154.3202, 'train_samples_per_second': 3.318, 'train_steps_per_second': 0.829, 'train_loss': 0.0, 'epoch': 1.0}
Time: 154.32
Samples/second: 3.32
GPU memory occupied: 6871 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


In [14]:
# BF16 Training
"""
    NVIDIA Ampere 장비나 더 최신의 H/W를 사용한다면 bf16 training이 가능하다.
    정밀도는 fp16보다 떨어지나 더 큰 범위를 가질 수 있다.
    
    - fp16 최대값: 65535
    - bf16 최대값: 3.39e+38
"""

clean_training(
	per_device_train_batch_size = 1,
	gradient_accumulation_steps = 4,
	gradient_checkpointing = True,
	float_precision = "bf16"
)

{'train_runtime': 154.6933, 'train_samples_per_second': 3.31, 'train_steps_per_second': 0.827, 'train_loss': 0.0, 'epoch': 1.0}
Time: 154.69
Samples/second: 3.31
GPU memory occupied: 6871 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


In [15]:
# TF32 Training
"""
    Ampere H/W는 tf32라는 개쩌는 data type을 사용한다.
    fp32와 동일한 범위(8bit)를 갖고 있으나 fp16과 같은 정밀도(10bit)를 갖는다.
    > total: sign(1 bit) + range(8 bit) + precision(10 bit) = 19bits
    
    코드에 아래 두 줄만 작성하면 pytorch가 자동으로 fp32를 tf32로 변환하여 훈련한다.
    ```python
    import torch
    torch.backends.cuda.matmul.allow_tf32 = True
    ```
    
    HuggingFace에서는 한 줄로 사용 가능하다.
    ```python
    tf32 = True
    ```
"""
clean_training(
	per_device_train_batch_size = 1,
	gradient_accumulation_steps = 4,
	gradient_checkpointing = True,
	float_precision = "bf16"
)

{'train_runtime': 154.9592, 'train_samples_per_second': 3.304, 'train_steps_per_second': 0.826, 'train_loss': 0.0, 'epoch': 1.0}
Time: 154.96
Samples/second: 3.30
GPU memory occupied: 6889 MB.
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB



## Optimizer

transformer model의 일반적인 optimizer는 Adam 또는 AdamW(Adam with weight decay)를 사용해왔다. 성능은 좋지만 추가적인 메모리 사용량이 있다.

HuggingFace trainer model은 --optim flag를 통해 다양한 Optimizer를 지원한다.


#### Adafactor

가중치 행렬의 각 요소에 대한 평균 대신, 집계된 정보만 저장하므로 메모리 사용량이 현저히 줄어듦.

단, 수렴이 Adam의 수렴보다 느린 경우가 있음

In [17]:
clean_training(
	per_device_train_batch_size = 4,
	optim = "adafactor"
)

{'train_runtime': 146.7754, 'train_samples_per_second': 3.488, 'train_steps_per_second': 0.872, 'train_loss': 0.0, 'epoch': 1.0}
Time: 146.78
Samples/second: 3.49
GPU memory occupied: 4771 MB.

_______________________torch.cuda.empty_cache()_______________________
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


메모리 사용량이 현저히 줄어든게 보인다.

앞의 다른 메스드들도 추가한 결과는 다음과 같다.

In [18]:
clean_training(
	per_device_train_batch_size = 1,
	gradient_checkpointing = True,
	gradient_accumulation_steps = 4,
	float_precision = "fp16",
	optim = "adafactor"
)

{'train_runtime': 160.8274, 'train_samples_per_second': 3.184, 'train_steps_per_second': 0.796, 'train_loss': 0.0, 'epoch': 1.0}
Time: 160.83
Samples/second: 3.18
GPU memory occupied: 4532 MB.

_______________________torch.cuda.empty_cache()_______________________
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


튜토리얼 보면 3배는 빨라지던데 왜징..

#### 8-bit Adam

Adafactor처럼 optimizer의 상태를 집계하는 대신, 8-bit Adam은 full state를 유지하고 양자화한다.

이는 낮은 정밀도로 저장했다가 최적화 시에만 원래 정밀도로 계산해 메모리 사용량을 줄일 수 있다.

Trainer에 플래그로 추가할 수 없어서 8-bit Adam optimizer(를 구현한 bitsandbytes)를 설치해 커스터마이징해야한다.

- Linux: [여기](https://github.com/TimDettmers/bitsandbytes)서 설치
- Windows: [여기](https://github.com/jllllll/bitsandbytes-windows-webui)서 설치

System Env.
- CONDA_PREFIX: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.8\bin
- 위 환경 변수는 아래 파일의 설치 위치
    - Linux: \*cuda\*.so
    - Windows: \*cuda\*.dll
    - ex.) cudart64_110.dll 설치 위치

설치 후 optimizer를 초기화해야 하는데 고려사항이 2개 있다.
1. model`s parameters를 가중치 감쇠(weight decay)를 적용할 그룹과 적용하지 않을 그룹으로 나눈다.
2. AdamW optimizer와 동일한 파라미터를 사용하기 위해 몇가지 전달인자에 대한 housekeeping이 필요하다.

In [15]:
clean_training(
    per_device_train_batch_size = 4,
    use_bnb = True
)

{'train_runtime': 101.0413, 'train_samples_per_second': 5.067, 'train_steps_per_second': 1.267, 'train_loss': 0.015763485804200172, 'epoch': 1.0}
Time: 101.04
Samples/second: 5.07
GPU memory occupied: 10429 MB.

_______________________torch.cuda.empty_cache()_______________________
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


In [16]:
# 모든 메서드 적용

clean_training(
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    gradient_checkpointing = True,
    float_precision = "fp16",
    use_bnb = True
)

{'train_runtime': 148.6336, 'train_samples_per_second': 3.445, 'train_steps_per_second': 0.861, 'train_loss': 0.0, 'epoch': 1.0}
Time: 148.63
Samples/second: 3.44
GPU memory occupied: 4889 MB.

_______________________torch.cuda.empty_cache()_______________________
Allocated GPU memory: 1.27GB
Reserved GPU memory: 1.30GB


얘도 메서드 적용 안한게 더 빠르네...

## Accelerate



## BetterTransformer: PyTorch-native transformer fastpath

Inference 전용 Transformer.

Transformer의 encoder, encoder layer, multi head attention을 다음 대표적인 두 방법으로 가속화.
1. fused kernel: 효율 up
2. exploiting sparsity in the inputs: pad_token과 같이 불필요한 부분을 생략

\* PyTorch 1.13 이상의 버전에서 호환
`pip install accelerate optimum`


======================gc======================
Allocated memory: 1.3GB
GPU Reserved memory: 4.0GB
======================empty cache======================
Allocated memory: 1.3GB
GPU Reserved memory: 1.3GB


TypeError: can only assign an iterable